In [7]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    WELFAKE
'''

data = pd.read_csv(r"../data/fake_news_conventional/WELFAKE_PROCESSED.csv")
data["flag"] = data["label"]
data

print("WELFAKE LOADED")


,Unnamed: 0.1,Unnamed: 0,title,text,label,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub,title_vader_comp,title_vader_neg,title_vader_neu,title_vader_pos,text_vader_neg,text_vader_neu,text_vader_pos,flag
0,35498,35498,Fox News Just Compared Slave Owners To Vict...,Nobody can accuse Fox News of being sane most ...,1,-0.073710,0.456448,-0.333333,0.666667,-0.4588,0.333,0.667,0.000,0.181,0.743,0.077,1
1,30850,30850,Rescuers search for Philippine storm victims a...,MANILA Reuters Rescuers in the Philippines...,0,0.031739,0.337790,0.000000,0.000000,-0.3182,0.204,0.796,0.000,0.182,0.746,0.072,0
2,65840,65840,Obama takes on role of America s pitchman at G...,HANOVER Germany Reuters U S President Ba...,0,0.290539,0.514975,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.009,0.904,0.087,0
3,65424,65424,ISRAEL WILL NAME New Train Station Near Wester...,Israel s transportation minister is pushing ah...,1,0.081429,0.257857,0.078788,0.284848,0.0000,0.000,1.000,0.000,0.024,0.896,0.080,1
4,21786,21786,Germanys Angela Merkel Makes Incredibly Naive ...,Perhaps Merkel should have considered the seri...,1,0.105782,0.422132,-0.300000,1.000000,-0.3384,0.138,0.862,0.000,0.119,0.764,0.117,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1247,1247,Terrorist Jon Ritzheimer Uses Sobbing Daughte...,Jon Ritzheimer infamous hater of Muslims and ...,1,0.053139,0.473593,0.000000,0.000000,-0.8074,0.378,0.622,0.000,0.109,0.801,0.090,1
4996,49972,49972,Bernie Sanders American Dream is in Denmark,Copenhagen Denmark CNN Open a newspaper on ...,0,0.084418,0.440525,0.000000,0.000000,0.2500,0.000,0.750,0.250,0.054,0.835,0.111,0
4997,1163,1163,Connecticut Becomes First State To Boycott Ind...,WASHINGTON Connecticut Gov Dan Malloy D ...,0,0.071160,0.467252,0.250000,0.333333,-0.3182,0.187,0.813,0.000,0.047,0.902,0.051,0
4998,15545,15545,Betting on a Trump win or a North Korea H bomb...,NEW YORK WELLINGTON Reuters Erik Duhaime i...,0,0.064652,0.339281,0.800000,0.400000,0.4588,0.143,0.580,0.277,0.043,0.864,0.093,0


In [8]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd


def get_tf_idf_vector(corpus: pd.Series) -> tuple[pd.DataFrame, list[str], TfidfVectorizer]:
    #temp = pd.concat((x, corpus), axis=1).reset_index()

    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=500, ngram_range=(1,3))
    tf_idf_features = vectorizer.fit_transform(corpus).toarray()

    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(tf_idf_features, columns=headers).reset_index(drop=True)
    return (feature_frame.reset_index(drop=True), names, vectorizer)



In [ ]:
from openpyxl import load_workbook, Workbook

'''
    Define filtered dataset, classes, features, dataframe for model accuracies, and excel file for results
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["flag"]

filtered["corpus"] = filtered.apply(lambda x: " ".join([x["title"], x["text"]]), axis=1)


# raw polarity and subjectivity scores from Textblob, Vader
features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]]



EXCEL_FILE = r"../data/fake_news_conventional/WELFAKE_RESULTS.xlsx"
# overwrite book if exists
book = Workbook()
book.save(filename=EXCEL_FILE)
book.close()


In [10]:
'''
    POLARITY + TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io



# average accuracies
comb_output = {"train" : None, "test" : None}
comb_matrices = []


comb_fpr = []
comb_tpr = []
comb_thresh = []

comb_auc = 0


train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    tf_idf_training, names, vectorizer = get_tf_idf_vector(filtered["corpus"].iloc[train_index])
    x_train = pd.concat((x.iloc[train_index].reset_index(drop=True), tf_idf_training), axis=1)
    
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = pd.DataFrame(vectorizer.transform(filtered["corpus"].iloc[test_index]).toarray(), columns=[f"__word{i}" for i in range(len(names))])
    x_test = pd.concat((x.iloc[test_index].reset_index(drop=True), x_test), axis=1)
    x_test = scaler.transform(x_test)
    
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    comb_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    comb_ensemble.fit(x_train, y_train)
    y_pred = comb_ensemble.predict(x_test)

    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    comb_fpr.append(fpr)
    comb_tpr.append(tpr)
    comb_thresh.append(thresh)
    comb_auc += metrics.auc(fpr, tpr)

    comb_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = comb_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

comb_output["test"] = [(test_total / run_count)]
comb_output["train"] = [(train_total / run_count)]
comb_output = pd.DataFrame(comb_output)

print("Average confusion matrix:")
comb_matrices = np.mean(np.array(comb_matrices), axis=0)
print(comb_matrices)


comb_fpr = np.mean(np.array(comb_fpr), axis=0)
comb_tpr = np.mean(np.array(comb_tpr), axis=0)
comb_thresh = np.mean(np.array(comb_thresh), axis=0)

comb_auc = comb_auc / float(run_count)


    


    


    

    
#x_test

comb_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.99595
Test accuracy on 5-fold cross-validation: 0.922
Average confusion matrix:
[[443.2  42.6]
 [ 35.4 478.8]]


,train,test
0,0.99595,0.922


In [11]:
'''
    POLARITY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
pol_output = {"train" : None, "test" : None}
pol_matrices = []


pol_fpr = []
pol_tpr = []
pol_thresh = []

pol_auc = 0

train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = x.iloc[train_index]
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = x.iloc[test_index]
    x_test = scaler.transform(x_test)
    
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    pol_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    pol_ensemble.fit(x_train, y_train)
    y_pred = pol_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    pol_fpr.append(fpr)
    pol_tpr.append(tpr)
    pol_thresh.append(thresh)
    pol_auc += metrics.auc(fpr, tpr)

    pol_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = pol_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

pol_output["test"] = [(test_total / run_count)]
pol_output["train"] = [(train_total / run_count)]
pol_output = pd.DataFrame(pol_output)

print("Average confusion matrix:")
pol_matrices = np.mean(np.array(pol_matrices), axis=0)
print(pol_matrices)

pol_fpr = np.mean(np.array(pol_fpr), axis=0)
pol_tpr = np.mean(np.array(pol_tpr), axis=0)
pol_thresh = np.mean(np.array(pol_thresh), axis=0)

pol_auc = pol_auc / float(run_count)




pol_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.8843
Test accuracy on 5-fold cross-validation: 0.701
Average confusion matrix:
[[341.  144.8]
 [154.2 360. ]]


,train,test
0,0.8843,0.701


In [12]:
'''
    TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
tf_output = {"train" : None, "test" : None}
tf_matrices = []

train_total = 0
test_total = 0
run_count = 0

tf_fpr = []
tf_tpr = []
tf_thresh = []

tf_auc = 0

#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    tf_idf_training, names, vectorizer = get_tf_idf_vector(filtered["corpus"].iloc[train_index])
    x_train = pd.concat((x.iloc[train_index].reset_index(drop=True), tf_idf_training), axis=1).drop(features, axis=1)
    
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = pd.DataFrame(vectorizer.transform(filtered["corpus"].iloc[test_index]).toarray(), columns=[f"__word{i}" for i in range(len(names))])
    x_test = pd.concat((x.iloc[test_index].reset_index(drop=True), x_test), axis=1).drop(features, axis=1)
    x_test = scaler.transform(x_test)
    
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    
    tf_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    tf_ensemble.fit(x_train, y_train)
    y_pred = tf_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    tf_fpr.append(fpr)
    tf_tpr.append(tpr)
    tf_thresh.append(thresh)
    tf_auc += metrics.auc(fpr, tpr)


    tf_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = tf_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

tf_output["test"] = [(test_total / run_count)]
tf_output["train"] = [(train_total / run_count)]
tf_output = pd.DataFrame(tf_output)

print("Average confusion matrix:")
tf_matrices = np.mean(np.array(tf_matrices), axis=0)
print(tf_matrices)

tf_fpr = np.mean(np.array(tf_fpr), axis=0)
tf_tpr = np.mean(np.array(tf_tpr), axis=0)
tf_thresh = np.mean(np.array(tf_thresh), axis=0)

tf_auc = tf_auc / float(run_count)



tf_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.99465
Test accuracy on 5-fold cross-validation: 0.9240000000000002
Average confusion matrix:
[[444.6  41.2]
 [ 34.8 479.4]]


,train,test
0,0.99465,0.924


In [13]:
'''
    combine accuracies into one table
'''

final_results = pd.concat((pol_output, tf_output, comb_output), axis=0)
final_results.index = ["Polarity", "Tf-Idf", "Polarity + Tf-Idf"]
final_results

,train,test
Polarity,0.88430,0.701
Tf-Idf,0.99465,0.924
Polarity + Tf-Idf,0.99595,0.922


In [14]:
'''
    Finally, save accuracy metrics to the spreadsheet
'''




book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book
final_results.to_excel(writer, sheet_name=f"accuracies")
book.save(filename=EXCEL_FILE)
book.close()



C:\Users\desmo\AppData\Local\Temp\ipykernel_12552\3534906820.py:10: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


In [15]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io


plt.figure()
lw = 2
plt.plot(pol_fpr, pol_tpr,
 lw=lw, label='Polarity: (%0.2f)' % pol_auc)
plt.plot(tf_fpr, tf_tpr,
 lw=lw, label='Tf-Idf: (%0.2f)' % tf_auc)
plt.plot(comb_fpr, comb_tpr,
 lw=lw, label='Polarity + Tf-Idf: (%0.2f)' % comb_auc)
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
#plt.show()


# save figure as PNG
png = io.BytesIO()
plt.savefig(png, format="png")


# write PNG to excel file
book = load_workbook(EXCEL_FILE)
ws = book.active

img = openpyxl.drawing.image.Image(png)
img.anchor = "A1"
ws.add_image(img)
book.save(filename=EXCEL_FILE)
plt.close()
book.close()

